## Homework#1-2
This is a second part of assignment #1. In this assignment, you are going to practice using `pandas` to learn about data. Please note that this assignment is not challenging and may not contain work from the demonstration file but carries extra work we may require to do in data preprocessing. 

Due Date: 9/16/2020 11:59 PM

### Problem 1
In this question, we will use the csv file **Temp.csv**. 
- Read it into a data frame, call it `Temp`. (Import the packages you think you need).

In [2]:
import pandas
import numpy as np
Temp = pandas.read_csv('Temp.csv')
#print(Temp.head())

- We learned that the temperature at New Haven, Connecticut is currently 58 degree fahrenheit. Add this information into the DataFrame.

In [3]:
Temp = Temp.append({'City': 'New Haven', 'State': 'Connecticut', 'Temperature': 58}, ignore_index=True)
Temp

,City,State,Temperature
0,NYC,New York,67
1,Albany,New York,49
2,Buffalo,New York,38
3,Hartford,Connecticut,63
4,Bridgeport,Connecticut,58
5,Treton,New Jersey,72
6,Newark,New Jersey,68
7,New Haven,Connecticut,58


- Replace the names of the states by their abbreviations. That is 
    - Connecticut --> CT
    - New Jersey  --> NJ
    - New York    --> NY

In [4]:
def abbreviations(state):
    if state == 'Connecticut':
        return 'CT'
    elif state == 'New Jersey':
        return 'NJ'
    elif state == 'New York':
        return 'NY'
    return state
Temp['State'] = Temp['State'].apply(abbreviations)
Temp

,City,State,Temperature
0,NYC,NY,67
1,Albany,NY,49
2,Buffalo,NY,38
3,Hartford,CT,63
4,Bridgeport,CT,58
5,Treton,NJ,72
6,Newark,NJ,68
7,New Haven,CT,58


- Select the rows in New York.

In [5]:
newYork = Temp[Temp['State'] == 'NY']
newYork

,City,State,Temperature
0,NYC,NY,67
1,Albany,NY,49
2,Buffalo,NY,38


- What is the average of the temperature of cities in NY?

In [6]:
newYork['Temperature'].sum()/len(newYork)

51.333333333333336

- Find the mean and the standard deviation of the temperature of each state.

In [7]:
spread = pandas.DataFrame(columns = ['Mean', 'Standard Deviation'], index = ['NY', 'CT', 'NJ'])
for x in spread.index:
    spread.loc[x, 'Standard Deviation'] = Temp[Temp['State'] == x]['Temperature'].std()
    spread.loc[x, 'Mean'] = Temp[Temp['State'] == x]['Temperature'].mean()
spread

,Mean,Standard Deviation
NY,51.3333,14.6401
CT,59.6667,2.88675
NJ,70,2.82843


- We know the capital of each state: 
    - Connecticut --> Hartford
    - New Jersey  --> Treton
    - New York    --> Albany
    
    Create a data frame, `Capital`, with two columns: State and Capital, which should look like:
    
|| State| Capital|
| :-: |:---: | :---:|
| 0| CT  | Hartford|
| 1| NJ | Treton|
| 2| NY| Albany|

In [8]:
Capital = pandas.DataFrame(np.array([['CT', 'Hartford'], ['NJ', 'Treton'], ['NY', 'Albany']]), columns = ['State', 'Capital'])
Capital

,State,Capital
0,CT,Hartford
1,NJ,Treton
2,NY,Albany


- Mutate a new boolean column, `Capital`, to `Temp`, which is `True` if the city in the row is the capital of the state, else `False`.

In [9]:
for x in Temp.index:
    Temp.loc[x, 'Capital'] = Temp.loc[x, 'City'] == Capital[Capital['State'] == Temp.loc[x, 'State']].values[0][1]
Temp

,City,State,Temperature,Capital
0,NYC,NY,67,False
1,Albany,NY,49,True
2,Buffalo,NY,38,False
3,Hartford,CT,63,True
4,Bridgeport,CT,58,False
5,Treton,NJ,72,True
6,Newark,NJ,68,False
7,New Haven,CT,58,False


### Problem 2

- a. Load the Gross Domestic Product data for the 190 ranked countries in the file **GDP.csv**. Load **Country.csv** into a DataFrame. 
- b. Match the data based on the country shortcode. How many of the IDs match? 
- c. Sort the data frame in descending order by GDP rank. What is the 13th country in the resulting data frame? (Note: GDP formats as 123,456,789.)
- d. What is the average GDP for the "High income: OECD" and "High income: nonOECD" group?
- e. What is the average GDP for different income groups and regions?

In [44]:
GDP = pandas.read_csv('GDP.csv', encoding='ISO-8859-1')
GDP = GDP.iloc[4:194, [0,1,3, 4]].reset_index(drop=True)
GDP.columns = ['Country', 'Ranking', 'Economy', 'GDP']
Country = pandas.read_csv('Country.csv', encoding='ISO-8859-1')

In [45]:
Join = GDP.merge(Country, left_on='Country', right_on='CountryCode')
print(len(Join))
Join['GDP'] = Join['GDP'].str.replace(',', '').astype(float)


189


In [46]:
Join = Join.sort_index(ascending = False).reset_index(drop = True)
Join.loc[12, ['Country', 'Economy']]


Country        GRD
Economy    Grenada
Name: 12, dtype: object

In [37]:
Income = pandas.DataFrame(columns = ['Section'])
Income['Section'] = Join['Income Group'].drop_duplicates().drop_duplicates()
def average(df, dataframe, col):
    return dataframe[dataframe[col] == df]['GDP'].mean()
Income['Mean'] = Income['Section'].apply(average, args = (Join, 'Income Group'))
print(Income.head())
"""
Income = pandas.DataFrame(columns = ['Section'])
Income['Section'] = Join['Income Group'].drop_duplicates().drop_duplicates()
Region = pandas.DataFrame(columns = ['Section'])
Region['Section'] = Join['Region'].drop_duplicates().drop_duplicates()
def average(df, dataframe, col):
    return dataframe[dataframe[col] == df]['GDP'].str.replace(',', '').astype(float).mean()
Income['Mean'] = Income['Section'].apply(average, args = (Join, 'Income Group'))
Region['Mean'] = Region['Section'].apply(average, args = (Join, 'Region'))
#print(Income.head())
#print(Region.head())
"""


                 Section          Mean
0    Lower middle income  2.566635e+05
3    Upper middle income  2.318478e+05
8             Low income  1.441078e+04
29  High income: nonOECD  1.043498e+05
67     High income: OECD  1.483917e+06


"\nIncome = pandas.DataFrame(columns = ['Section'])\nIncome['Section'] = Join['Income Group'].drop_duplicates().drop_duplicates()\nRegion = pandas.DataFrame(columns = ['Section'])\nRegion['Section'] = Join['Region'].drop_duplicates().drop_duplicates()\ndef average(df, dataframe, col):\n    return dataframe[dataframe[col] == df]['GDP'].str.replace(',', '').astype(float).mean()\nIncome['Mean'] = Income['Section'].apply(average, args = (Join, 'Income Group'))\nRegion['Mean'] = Region['Section'].apply(average, args = (Join, 'Region'))\n#print(Income.head())\n#print(Region.head())\n"

In [38]:
pd = Join.groupby(["Income Group", "Region"]).agg({'GDP':'mean'})
print(pd)

                                                          GDP
Income Group         Region                                  
High income: OECD    East Asia & Pacific         2.197268e+06
                     Europe & Central Asia       7.567044e+05
                     Middle East & North Africa  2.582170e+05
                     North America               9.033012e+06
High income: nonOECD East Asia & Pacific         1.496240e+05
                     Europe & Central Asia       2.776660e+04
                     Latin America & Caribbean   2.795480e+04
                     Middle East & North Africa  2.142531e+05
                     North America               5.474000e+03
                     Sub-Saharan Africa          1.769700e+04
Low income           East Asia & Pacific         8.154667e+03
                     Europe & Central Asia       6.723500e+03
                     Latin America & Caribbean   7.843000e+03
                     South Asia                  5.193833e+04
        